# Project1 - getting to know the data

## Information from website
>A retrospective sample of males in a heart-disease high-risk region
of the Western Cape, South Africa. There are roughly two controls per
case of CHD. Many of the CHD positive men have undergone blood
pressure reduction treatment and other programs to reduce their risk
factors after their CHD event. In some cases the measurements were
made after these treatments. These data are taken from a larger
dataset, described in  Rousseauw et al, 1983, South African Medical
Journal. 

**sbp**		    systolic blood pressure 
* The highest pressure when your heart beats and pushes the blood round your body

**tobacco**		cumulative tobacco (kg)

**ldl**  		low densiity lipoprotein cholesterol 
* Lipoproteins transfer lipids (fats) around the body in the extracellular fluid thereby facilitating fats to be available and taken up by the cells body wide via receptor-mediated endocytosis.

**adiposity**   (fedme)
The body adiposity index (BAI) is a method of measuring the **amount of body fat in humans**. The BAI is **calculated without using body weight**, unlike the body mass index (BMI). Instead, it uses the size of the hips compared to the person's height.
Based on population studies, the **BAI is approximately equal to the percentage of body fat for adult men and women of differing ethnicities**.[1]

**famhist**		family history of heart disease (Present, Absent)

**typea**		type-A behavior
* Type A and Type B personality theory describes two contrasting personality types. In this theory, personalities that are more competitive, outgoing, ambitious, impatient and/or aggressive are labeled Type A, while more relaxed personalities are labeled Type B.
* The two cardiologists who developed this theory came to believe that Type A personalities had a greater chance of developing coronary heart disease. Following the results of further studies and considerable controversy about the role of the tobacco industry funding of early research in this area, some reject, either partially or completely, the link between Type A personality and coronary disease. Nevertheless, this research had a significant effect on the development of the health psychology field, in which psychologists look at how an individual's mental state affects their physical health

**obesity**
* BMI-measure?

**alcohol**		current alcohol consumption

**age**		    age at onset

**chd**		    response, coronary heart disease


## Preparation of data 

In [18]:
# Getting relevant packages
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import urllib2
from collections import Counter 
import numpy as np
from operator import itemgetter
from scipy import linalg
import geoplotlib as gpl

In [19]:
# Reading the data into python from webpage using pandas
disease_data = pd.read_csv('SAheart.csv')
# Remove redundant column with "row names"
disease_data.drop(disease_data.columns[[0]], axis=1, inplace=True)

In [20]:
import io
import requests
url    = "https://statweb.stanford.edu/~tibs/ElemStatLearn/datasets/SAheart.data"
source = requests.get(url).content
data   = pd.read_csv(io.StringIO(source.decode('utf-8')))

# Remove redundant column with "row names"
#data.drop(data.columns[[0]], axis=1, inplace=True)

In [44]:
print max(data.alcohol)
print min(data.alcohol)

147.19
0.0


In [25]:
data.columns

Index([u'row.names', u'sbp', u'tobacco', u'ldl', u'adiposity', u'famhist',
       u'typea', u'obesity', u'alcohol', u'age', u'chd'],
      dtype='object')

### Reading directly from url into pandas dataframe

In [17]:
al= data.iloc[:,8].values
max(al)

147.19

In [15]:
min(ob)

14.699999999999999